- 연남동 필터링 : 주소지에 '연남동' 포함된 행 
- 우편번호 + 사업장명(가게명) 열 생성하여 해당 데이터로 구글maps 검색  

In [ ]:
import pandas as pd
import requests
import time
from tqdm import tqdm

# Google API 키
API_KEY = 

In [17]:
# 1단계: csv 파일 불러오기
df = pd.read_csv("C:/medici_class/00.프로젝트/최종프로젝트 임시파일/마포구_음식점.csv", dtype={"도로명우편번호": str}, encoding='cp949')
df.head()

,번호,개방서비스명,개방서비스아이디,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,...,공장판매직직원수,공장생산직직원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
0,1,일반음식점,07_24_04_P,3130000,3130000-101-2025-00327,2025-05-14,NaN,1,영업/정상,1,...,0.0,0.0,NaN,0.0,0.0,N,54.75,NaN,NaN,NaN
1,2,일반음식점,07_24_04_P,3130000,3130000-101-2025-00326,2025-05-14,NaN,1,영업/정상,1,...,0.0,0.0,NaN,0.0,0.0,N,62.04,NaN,NaN,NaN
2,3,일반음식점,07_24_04_P,3130000,3130000-101-2025-00322,2025-05-13,NaN,1,영업/정상,1,...,0.0,0.0,NaN,0.0,0.0,N,74.41,NaN,NaN,NaN
3,4,일반음식점,07_24_04_P,3130000,3130000-101-2025-00323,2025-05-13,NaN,1,영업/정상,1,...,0.0,0.0,NaN,0.0,0.0,N,65.00,NaN,NaN,NaN
4,5,일반음식점,07_24_04_P,3130000,3130000-101-2025-00325,2025-05-13,NaN,1,영업/정상,1,...,0.0,0.0,NaN,0.0,0.0,N,82.80,NaN,NaN,NaN


In [18]:
# 주소지가 적힌 열 2개
address_columns = ['소재지전체주소', '도로명전체주소']

# 연남동이 포함된 행 필터링
mask = df[address_columns[0]].astype(str).str.contains("연남동") | df[address_columns[1]].astype(str).str.contains("연남동")
df_yeonnam = df[mask].copy()
df_yeonnam

,번호,개방서비스명,개방서비스아이디,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,...,공장판매직직원수,공장생산직직원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
2,3,일반음식점,07_24_04_P,3130000,3130000-101-2025-00322,2025-05-13,NaN,1,영업/정상,1,...,0.0,0.0,NaN,0.0,0.0,N,74.41,NaN,NaN,NaN
15,16,일반음식점,07_24_04_P,3130000,3130000-101-2025-00313,2025-05-12,NaN,1,영업/정상,1,...,0.0,0.0,NaN,0.0,0.0,N,50.37,NaN,NaN,NaN
16,17,일반음식점,07_24_04_P,3130000,3130000-101-2025-00299,2025-05-09,NaN,1,영업/정상,1,...,0.0,0.0,NaN,0.0,0.0,N,73.17,NaN,NaN,NaN
20,21,일반음식점,07_24_04_P,3130000,3130000-101-2025-00298,2025-05-09,NaN,1,영업/정상,1,...,0.0,0.0,NaN,0.0,0.0,N,45.35,NaN,NaN,NaN
47,48,일반음식점,07_24_04_P,3130000,3130000-101-2025-00282,2025-05-07,NaN,1,영업/정상,1,...,0.0,0.0,NaN,0.0,0.0,N,35.80,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8056,8057,일반음식점,07_24_04_P,3130000,3130000-101-1992-05215,1992-03-01,NaN,1,영업/정상,1,...,NaN,NaN,NaN,NaN,NaN,N,24.32,NaN,NaN,NaN
8081,8082,일반음식점,07_24_04_P,3130000,3130000-101-1990-01963,1990-11-21,NaN,1,영업/정상,1,...,0.0,0.0,NaN,0.0,0.0,N,36.73,NaN,NaN,NaN
8121,8122,일반음식점,07_24_04_P,3130000,3130000-101-1987-02386,1987-05-09,NaN,1,영업/정상,1,...,0.0,0.0,NaN,0.0,0.0,N,31.91,NaN,NaN,NaN
8136,8137,일반음식점,07_24_04_P,3130000,3130000-101-1985-05216,1985-09-21,NaN,1,영업/정상,1,...,NaN,NaN,NaN,NaN,NaN,N,61.91,NaN,NaN,NaN


In [19]:
df_yeonnam.columns

Index(['번호', '개방서비스명', '개방서비스아이디', '개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자',
       '영업상태구분코드', '영업상태명', '상세영업상태코드', '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자',
       '재개업일자', '소재지전화', '소재지면적', '소재지우편번호', '소재지전체주소', '도로명전체주소', '도로명우편번호',
       '사업장명', '최종수정시점', '데이터갱신구분', '데이터갱신일자', '업태구분명', '좌표정보X(EPSG5174)',
       '좌표정보Y(EPSG5174)', '위생업태명', '남성종사자수', '여성종사자수', '영업장주변구분명', '등급구분명',
       '급수시설구분명', '총직원수', '본사직원수', '공장사무직직원수', '공장판매직직원수', '공장생산직직원수',
       '건물소유구분명', '보증액', '월세액', '다중이용업소여부', '시설총규모', '전통업소지정번호', '전통업소주된음식',
       '홈페이지'],
      dtype='object')

In [20]:
# 불필요한 열 삭제
df_yeonnam = df_yeonnam.drop(['번호', '개방서비스명', '개방서비스아이디', '개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자',
       '영업상태구분코드', '영업상태명', '상세영업상태코드', '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자',
       '재개업일자', '소재지전화', '소재지면적', '소재지우편번호', '최종수정시점', '데이터갱신구분', '데이터갱신일자', '업태구분명', '좌표정보X(EPSG5174)',
       '좌표정보Y(EPSG5174)', '위생업태명', '남성종사자수', '여성종사자수', '영업장주변구분명', '등급구분명',
       '급수시설구분명', '총직원수', '본사직원수', '공장사무직직원수', '공장판매직직원수', '공장생산직직원수',
       '건물소유구분명', '보증액', '월세액', '다중이용업소여부', '시설총규모', '전통업소지정번호', '전통업소주된음식',
       '홈페이지'], axis=1)
df_yeonnam

,소재지전체주소,도로명전체주소,도로명우편번호,사업장명
2,서울특별시 마포구 연남동 383-100,"서울특별시 마포구 연희로1길 63, 2층 (연남동)",03982,사르르
15,서울특별시 마포구 연남동 390-71,"서울특별시 마포구 동교로38길 13, 지1층 (연남동)",03982,빨강꼬치&꼬떡상회
16,서울특별시 마포구 연남동 373-12,"서울특별시 마포구 연남로 8, 1층 (연남동)",03989,지라파
20,서울특별시 마포구 연남동 373-12,"서울특별시 마포구 연남로 8, 지층 (연남동)",03989,지라파
47,서울특별시 마포구 연남동 390-28,"서울특별시 마포구 동교로38길 27-19, 지층 B02호 (연남동)",03982,돈토키
...,...,...,...,...
8056,서울특별시 마포구 연남동 228-5,"서울특별시 마포구 성미산로 186 (연남동, 1층)",03982,연남제비
8081,서울특별시 마포구 연남동 260-28 1층,"서울특별시 마포구 동교로 240 (연남동,1층)",03982,유키모찌 연남점
8121,서울특별시 마포구 연남동 568-23 (1층),"서울특별시 마포구 동교로27길 41 (연남동,(1층))",03991,항저우 샤롱바오
8136,서울특별시 마포구 연남동 228-1,"서울특별시 마포구 성미산로 190 (연남동, 1층)",03982,뭉텅 연남점


In [21]:
df_yeonnam.info()

<class 'pandas.core.frame.DataFrame'>
Index: 807 entries, 2 to 8148
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   소재지전체주소  804 non-null    object
 1   도로명전체주소  807 non-null    object
 2   도로명우편번호  806 non-null    object
 3   사업장명     807 non-null    object
dtypes: object(4)
memory usage: 31.5+ KB


In [22]:
# 우편번호 + 가게명 조합 열 생성
df_yeonnam["검색어"] = df_yeonnam["도로명우편번호"] + " " + df_yeonnam["사업장명"]
df_yeonnam

,소재지전체주소,도로명전체주소,도로명우편번호,사업장명,검색어
2,서울특별시 마포구 연남동 383-100,"서울특별시 마포구 연희로1길 63, 2층 (연남동)",03982,사르르,03982 사르르
15,서울특별시 마포구 연남동 390-71,"서울특별시 마포구 동교로38길 13, 지1층 (연남동)",03982,빨강꼬치&꼬떡상회,03982 빨강꼬치&꼬떡상회
16,서울특별시 마포구 연남동 373-12,"서울특별시 마포구 연남로 8, 1층 (연남동)",03989,지라파,03989 지라파
20,서울특별시 마포구 연남동 373-12,"서울특별시 마포구 연남로 8, 지층 (연남동)",03989,지라파,03989 지라파
47,서울특별시 마포구 연남동 390-28,"서울특별시 마포구 동교로38길 27-19, 지층 B02호 (연남동)",03982,돈토키,03982 돈토키
...,...,...,...,...,...
8056,서울특별시 마포구 연남동 228-5,"서울특별시 마포구 성미산로 186 (연남동, 1층)",03982,연남제비,03982 연남제비
8081,서울특별시 마포구 연남동 260-28 1층,"서울특별시 마포구 동교로 240 (연남동,1층)",03982,유키모찌 연남점,03982 유키모찌 연남점
8121,서울특별시 마포구 연남동 568-23 (1층),"서울특별시 마포구 동교로27길 41 (연남동,(1층))",03991,항저우 샤롱바오,03991 항저우 샤롱바오
8136,서울특별시 마포구 연남동 228-1,"서울특별시 마포구 성미산로 190 (연남동, 1층)",03982,뭉텅 연남점,03982 뭉텅 연남점


In [27]:
# Google Maps API 요청 함수
def get_place_id(query):
    """검색어로 place_id 얻기"""
    url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    params = {
        "input": query,
        "inputtype": "textquery",
        "fields": "place_id,name",
        "key": API_KEY
    }
    res = requests.get(url, params=params).json()
    candidates = res.get("candidates")
    if candidates:
        return candidates[0]["place_id"], candidates[0]["name"]
    return None, None

def get_reviews(place_id):
    """place_id로부터 평점과 리뷰내용 가져오기"""
    url = "https://maps.googleapis.com/maps/api/place/details/json"
    params = {
        "place_id": place_id,
        "fields": "review",
        "key": API_KEY,
        "language": "ko"
    }
    res = requests.get(url, params=params).json()
    reviews = res.get("result", {}).get("reviews", [])
    result = []
    for r in reviews:
        result.append({
            "별점": r.get("rating"),
            "리뷰내용": r.get("text"),
            "작성자": r.get("author_name"),
            "작성자 프로필 링크":r.get("author_url"),
            "시간": r.get("relative_time_description")
        })
    return result

# 리뷰 수집
all_reviews = []
for i, row in tqdm(df_yeonnam.iterrows(), total=len(df_yeonnam)):
    search_query = row["검색어"]
    store_name = row["사업장명"]
    
    try:
        place_id, place_name = get_place_id(search_query)
        if place_id:
            reviews = get_reviews(place_id)
            for review in reviews:
                review["사업장명"] = store_name
                review["검색결과 가게명"] = place_name
                all_reviews.append(review)
        else:
            print(f"[미발견] {search_query}")
        time.sleep(0.3)  # 너무 빠른 요청 방지
    except Exception as e:
        print(f"[오류] {search_query}: {e}")
        continue

# 결과 저장
df_reviews = pd.DataFrame(all_reviews)
df_reviews.to_csv("yeonnam_restaurant_reviews_googlemaps.csv", index=False, encoding="utf-8-sig")
print("리뷰 수집 완료 및 저장 완료")


  0%|          | 1/807 [00:00<12:36,  1.07it/s]

[미발견] 03982 빨강꼬치&꼬떡상회


  0%|          | 2/807 [00:01<10:05,  1.33it/s]

[미발견] 03989 지라파


  0%|          | 3/807 [00:02<09:23,  1.43it/s]

...

[미발견] 03974 만경식당


 98%|█████████▊| 792/807 [13:13<00:15,  1.03s/it]

[미발견] 03987 김밥세상


100%|██████████| 807/807 [13:29<00:00,  1.00s/it]

리뷰 수집 완료 및 저장 완료


In [6]:
df_yeonnam_reviews_gmaps = pd.read_csv("C:/medici_class/00.프로젝트/최종프로젝트 임시파일/yeonnam_restaurant_reviews_googlemaps.csv", encoding='cp949')
df_yeonnam_reviews_gmaps

,별점,리뷰내용,작성자,작성자 프로필 링크,시간,사업장명,검색결과 가게명
0,5,대박 사르르 녹는 뚱카롱 짱맛 꿀맛이에용ㅠㅠ,JiWon Kim,https://www.google.com/maps/contrib/1055269768...,6년 전,사르르,사르르과자점
1,5,NaN,두리,https://www.google.com/maps/contrib/1122504793...,4년 전,사르르,사르르과자점
2,5,히레와 로스 카츠 정식 점심으로 먹었는데 제주 흑돼지 사용과 말돈소금 및 비법 드레...,우동희,https://www.google.com/maps/contrib/1048404036...,4달 전,돈토키,카마카츠
3,5,남자친구랑 홍대에서 돈카츠맛집을 찾았습니다!!!\r\n분위기도 너무 좋아서 나중에 ...,박지현,https://www.google.com/maps/contrib/1130968599...,6달 전,돈토키,카마카츠
4,4,밥이 솥밥으로 나온다는게 매력적임. 돈까스는 두툼하고 지방이 적당하게 있어서 맛있었...,투어리즘,https://www.google.com/maps/contrib/1093313547...,2년 전,돈토키,카마카츠
...,...,...,...,...,...,...,...
2796,3,요즘엔 인스타에서 뜨는 광고를 통해 신상 가게 정보를 얻는 좋은 예도 있지만 여기저...,ethan ghymn,https://www.google.com/maps/contrib/1042004016...,3년 전,항저우 샤롱바오,샤오롱바오
2797,5,연트럴파크 맛집 뭉텅 연남점을 찾았습니다. 연남동 고기집 뭉텅은 홍대입구역 3번 출...,호빵공장,https://www.google.com/maps/contrib/1023673874...,10달 전,뭉텅 연남점,뭉텅 연남점
2798,5,본점보다맛있어요! 뭉텅여기서 처음가고 너무맛있어서 본점갔는데 연남점이 훨씬고기가 큰...,Junwon Choi,https://www.google.com/maps/contrib/1131549655...,7달 전,뭉텅 연남점,뭉텅 연남점
2799,5,오랜만에 맛있는고기집 찾았어요 고기찾아먹진않는타입인데 가끔땡길때 있잖아유?\r\n사...,하릴없이,https://www.google.com/maps/contrib/1162235992...,1년 전,뭉텅 연남점,뭉텅 연남점


In [7]:
df_yeonnam_reviews_gmaps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2801 entries, 0 to 2800
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   별점          2801 non-null   int64 
 1   리뷰내용        2704 non-null   object
 2   작성자         2801 non-null   object
 3   작성자 프로필 링크  2801 non-null   object
 4   시간          2801 non-null   object
 5   사업장명        2801 non-null   object
 6   검색결과 가게명    2801 non-null   object
dtypes: int64(1), object(6)
memory usage: 153.3+ KB


In [8]:
df_yeonnam_reviews_gmaps['사업장명'].value_counts()

사업장명
시실리             10
향미              10
리틀파파포           10
고스트블랙 연남점       10
해피치즈스마일 연남점      5
                ..
소코               1
마스타 하이볼 연남점      1
굽네치킨 연남점         1
플루밍(Pluming)     1
제일 연남            1
Name: count, Length: 633, dtype: int64

In [9]:
df_yeonnam_reviews_gmaps['사업장명'].nunique()

633

In [10]:
from difflib import SequenceMatcher

def get_similarity(a, b):
    return SequenceMatcher(None, str(a), str(b)).ratio()  # 0~1 사이 유사도 반환

# 새로운 열 '이름_유사도' 추가
df_yeonnam_reviews_gmaps['검색결과와 사업장명 유사도'] = df_yeonnam_reviews_gmaps.apply(lambda row: get_similarity(row['사업장명'], row['검색결과 가게명']), axis=1)
df_yeonnam_reviews_gmaps

,별점,리뷰내용,작성자,작성자 프로필 링크,시간,사업장명,검색결과 가게명,검색결과와 사업장명 유사도
0,5,대박 사르르 녹는 뚱카롱 짱맛 꿀맛이에용ㅠㅠ,JiWon Kim,https://www.google.com/maps/contrib/1055269768...,6년 전,사르르,사르르과자점,0.666667
1,5,NaN,두리,https://www.google.com/maps/contrib/1122504793...,4년 전,사르르,사르르과자점,0.666667
2,5,히레와 로스 카츠 정식 점심으로 먹었는데 제주 흑돼지 사용과 말돈소금 및 비법 드레...,우동희,https://www.google.com/maps/contrib/1048404036...,4달 전,돈토키,카마카츠,0.000000
3,5,남자친구랑 홍대에서 돈카츠맛집을 찾았습니다!!!\r\n분위기도 너무 좋아서 나중에 ...,박지현,https://www.google.com/maps/contrib/1130968599...,6달 전,돈토키,카마카츠,0.000000
4,4,밥이 솥밥으로 나온다는게 매력적임. 돈까스는 두툼하고 지방이 적당하게 있어서 맛있었...,투어리즘,https://www.google.com/maps/contrib/1093313547...,2년 전,돈토키,카마카츠,0.000000
...,...,...,...,...,...,...,...,...
2796,3,요즘엔 인스타에서 뜨는 광고를 통해 신상 가게 정보를 얻는 좋은 예도 있지만 여기저...,ethan ghymn,https://www.google.com/maps/contrib/1042004016...,3년 전,항저우 샤롱바오,샤오롱바오,0.615385
2797,5,연트럴파크 맛집 뭉텅 연남점을 찾았습니다. 연남동 고기집 뭉텅은 홍대입구역 3번 출...,호빵공장,https://www.google.com/maps/contrib/1023673874...,10달 전,뭉텅 연남점,뭉텅 연남점,1.000000
2798,5,본점보다맛있어요! 뭉텅여기서 처음가고 너무맛있어서 본점갔는데 연남점이 훨씬고기가 큰...,Junwon Choi,https://www.google.com/maps/contrib/1131549655...,7달 전,뭉텅 연남점,뭉텅 연남점,1.000000
2799,5,오랜만에 맛있는고기집 찾았어요 고기찾아먹진않는타입인데 가끔땡길때 있잖아유?\r\n사...,하릴없이,https://www.google.com/maps/contrib/1162235992...,1년 전,뭉텅 연남점,뭉텅 연남점,1.000000


In [11]:
df_yeonnam_reviews_gmaps.to_csv("gmaps_reviews.csv", index=False, encoding="utf-8-sig")
print("리뷰 수집 완료 및 저장 완료")

리뷰 수집 완료 및 저장 완료
